In [1]:
%matplotlib inline
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
import seaborn as sns
import math as mt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import os

label=LabelEncoder()
# Any results you write to the current directory are saved as output.
sample= pd.read_csv("/kaggle/input/mai-ml-2019-linear-classification/sample_submission.csv")
train= pd.read_csv("/kaggle/input/mai-ml-2019-linear-classification/train.csv")
test= pd.read_csv("/kaggle/input/mai-ml-2019-linear-classification/test.csv")
full_data = [train, test]
for data in full_data:
    data['Fum']=data['SibSp']+data['Parch']
    data['Sex']=data['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    data['ChAge']=data["Age"].apply(lambda x: 0 if mt.isnan(x) else 1)
    data['Cabin'] = data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    
    data['Age'][np.isnan(data['Age'])]=data['Age'].mean()
    avg = data['Age'].mean()
    std = data['Age'].std()
    null_count = data['Age'].isnull().sum()
    null_random_list = np.random.randint(avg - std, avg + std, size=null_count)
    
    data['Age'][np.isnan(data['Age'])] = null_random_list
    
    data['ChFare']=data["Fare"].apply(lambda x: 0 if mt.isnan(x) else 1)
    data['Name'] = data['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
    data['Name'] = label.fit_transform(data['Name'])
    
    avg = data['Fare'].mean()
    
    std = data['Fare'].std()
    null_count = data['Fare'].isnull().sum()
    null_random_list = np.random.randint(avg - std, avg + std, size=null_count)
    
    data['Fare'][np.isnan(data['Fare'])] = null_random_list
    data['Embarked1']=data['Embarked'].apply(lambda x: 0 if x == 'C' else 1)
    data['Embarked']=data['Embarked'].apply(lambda x: 0 if x == 'Q' else 1)
    data['Fare'] = pd.qcut(data['Fare'], 4)
    data['Age'] = pd.cut(data['Age'].astype(int), 5)
    data['Age'] = label.fit_transform(data['Age'])
    data['Fare'] = label.fit_transform(data['Fare'])
    
drop_elements = [ 'Ticket','PassengerId']
print(train.head())
test1 = test.drop(drop_elements, axis = 1)
train1 = train.drop(drop_elements, axis = 1)
y_train = train1['Survived'].ravel()
x_train = train1.drop(['Survived'], axis=1).values

x_test=test1.values
'''from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)'''

clf = LogisticRegression()
params = {'penalty':['l1','l2'],'C':list(range(1,100,1)),'random_state':list(range(1,100,1))}
logi = GridSearchCV(clf, param_grid= params, cv = 5,scoring = "roc_auc",verbose=1)
logi.fit(x_train, y_train)

result = logi.predict(x_test)
test['Survived']=result

#sns.countplot(x = 'Tmp', data = train, hue = 'Survived')
#sns.countplot(x = 'Tmp', data = test, hue = 'Survived')
#print(train.head())
sample=test[['PassengerId','Survived']]
sample.to_csv('sample.csv', index=False)


NameError: name 'LabelEncoder' is not defined